# Introduction to Numba for CPU
[Numba](http://numba.pydata.org/) is an open source just-in-time (JIT) compiler that translates a subset of Python and NumPy code into fast machine code.

In [ ]:
import numpy as np
from numba import jit, njit, prange, vectorize
from concurrent.futures import ThreadPoolExecutor
import dask
import dask.delayed

In [ ]:
def slow_function(a,b):
    assert a.shape == b.shape
    c = np.empty(a.shape)
    for i in range(a.shape[0]):
        c[i] = a[i] @ b[i]
    return c

def fast_function(a,b):
    return njit()(slow_function)

# when using the njit decorator, the original function is accesable throught slow_function.py_func

In [ ]:
n=5
N=10000
a = np.random.rand(N,n,n)
b = np.random.rand(N,n,n)

%timeit slow_function(a,b)
# first call invokes the jit compiler and the second should be much faster
%timeit fast_function(a,b)
%timeit fast_function(a,b)

## Universal functions `ufuncs`
Universal functions are functions that broadcast an elementwise operation across input arrays of varying numbers of dimensions.

In [ ]:
@njit
def elementwise_operations(a,b):
    return a**2 + b**2

@vectorize(nopython=True)
def elementwise_operations_v(a,b):
    return a**2 + b**2

elementwise_operations.py_func(3,4) # call the interpreted function
elementwise_operations(3,4) # call the compiled function
elementwise_operations(np.ones((10,10)),np.ones((10,10)))
elementwise_operations_v(np.ones((10,10)),np.ones((10,10)))
# elementwise_operations.inspect_types() # check arg types of the compiled function
# elementwise_operations.inspect_types(pretty=True) # check arg types of the compiled function
elementwise_operations.signatures

## External multithreading
To combine with python threads, it is usefull to release the __global interpreter lock__

In [ ]:
@njit(nogil=True)
def do_some_computations():
    z=0
    for x in range(10000):
        y = x * np.random.randint(0,2)/2
        if x > y:
            z+= np.exp(x) + np.sin(y)
        else:
            z+= np.exp(y) + np.sin(x)
    return z

# compile
_ = do_some_computations()
delayed_do_some_computations = dask.delayed(do_some_computations)

In [ ]:
%%time
with ThreadPoolExecutor(8) as ex:
    ex.map(do_some_computations)

In [ ]:
%%time
futures = delayed_do_some_computations()
results = dask.compute(futures,num_workers=8)[0]

## Explicit multithreading using `prange()`

In [ ]:
@njit(nogil=True, parallel=True)
def run_in_parallel(end=10000):
    for x in prange(int(end)):
        do_some_computations()

run_in_parallel()
%time run_in_parallel()

In [ ]:
# function with reduction
@jit(nopython=True, parallel=True)
def monte_carlo_pi_parallel(nsamples):
    acc = 0
    # Only change is here
    for i in prange(nsamples):
        x = np.random.random()
        y = np.random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples
# acc is accessed in a thread-safe way [will get a race condition if acc is an array]
# Numba automatically initializes the random number generator in each thread independently

## Automatic multithreading of `Numpy` expressions

In [ ]:
@jit(nopython=True, parallel=True)
def computations_on_arrays(x,y):
        return np.exp(x) * np.sin(y)
x = np.random.uniform(-1, 1, size=1000000)
y = np.random.uniform(-1, 1, size=1000000)

computations_on_arrays_nothread = jit(nopython=True)(computations_on_arrays.py_func)

%timeit computations_on_arrays.py_func(x,y)
%timeit computations_on_arrays_nothread(x,y)
%timeit computations_on_arrays(x,y)

## SIMD Autovectorisation
These instructions operate on as many values as will fit into an input register. For AVX instructions, either 8 float32 values or 4 float64 values can be processed as a single input.

On x86_64, the name of the registers used indicates which level of SIMD is in use:

* SSE: `xmmX`
* AVX/AVX2: `ymmX`
* AVX-512: `zmmX`

where X is an integer.

In x86_64 assembly, SSE uses `subps` for "subtraction packed single precision" (AVX uses `vsubps`), representing vector float32 operations.  The `subpd` instruction (AVX = `vsubpd`) stands for "subtraction packed double precision", representing float64 operations.

ref: https://hub.gke.mybinder.org/user/numba-numba-examples-k9r91v25/notebooks/notebooks/simd.ipynb

In [ ]:
def find_instr(func, keyword, sig=0, limit=5):
    count = 0
    for l in func.inspect_asm(func.signatures[sig]).split('\n'):
        if keyword in l:
            count += 1
            print(l)
            if count >= limit:
                break
    if count == 0:
        print('No instructions found')

@jit(nopython=True)
def sqdiff(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        out[i] = (x[i] - y[i])**2
    return out

x32 = np.linspace(1, 2, 10000, dtype=np.float32)
y32 = np.linspace(2, 3, 10000, dtype=np.float32)
sqdiff(x32, y32)

x64 = x32.astype(np.float64)
y64 = y32.astype(np.float64)
sqdiff(x64, y64)

%timeit sqdiff(x32, y32)
%timeit sqdiff(x64, y64)

print('float32:')
find_instr(sqdiff, keyword='subp', sig=0)
print('---\nfloat64:')
find_instr(sqdiff, keyword='subp', sig=1)

## Other options
`@jit(nopython=True, fastmath=True)`
`find_instr(do_sum, keyword='mulp')`

import sys
# sys.getsizeof(np.ones(3))
import scipy.sparse as sc
sc.csr_matrix(np.ones(3)).data.nbytes

In [ ]:
@njit(nogil=True, parallel=True)
def run_in_parallel(end=10000):
    for x in prange(int(end)):
        do_some_computations()

run_in_parallel()
%time run_in_parallel()

In [ ]:
# function with reduction
@jit(nopython=True, parallel=True)
def monte_carlo_pi_parallel(nsamples):
    acc = 0
    # Only change is here
    for i in prange(nsamples):
        x = np.random.random()
        y = np.random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples
# acc is accessed in a thread-safe way [will get a race condition if acc is an array]
# Numba automatically initializes the random number generator in each thread independently

## Automatic multithreading of `Numpy` expressions

In [ ]:
@jit(nopython=True, parallel=True)
def computations_on_arrays(x,y):
        return np.exp(x) * np.sin(y)
x = np.random.uniform(-1, 1, size=1000000)
y = np.random.uniform(-1, 1, size=1000000)

computations_on_arrays_nothread = jit(nopython=True)(computations_on_arrays.py_func)

%timeit computations_on_arrays.py_func(x,y)
%timeit computations_on_arrays_nothread(x,y)
%timeit computations_on_arrays(x,y)

## SIMD Autovectorisation
These instructions operate on as many values as will fit into an input register. For AVX instructions, either 8 float32 values or 4 float64 values can be processed as a single input.

On x86_64, the name of the registers used indicates which level of SIMD is in use:

* SSE: `xmmX`
* AVX/AVX2: `ymmX`
* AVX-512: `zmmX`

where X is an integer.

In x86_64 assembly, SSE uses `subps` for "subtraction packed single precision" (AVX uses `vsubps`), representing vector float32 operations.  The `subpd` instruction (AVX = `vsubpd`) stands for "subtraction packed double precision", representing float64 operations.

ref: https://hub.gke.mybinder.org/user/numba-numba-examples-k9r91v25/notebooks/notebooks/simd.ipynb

In [ ]:
def find_instr(func, keyword, sig=0, limit=5):
    count = 0
    for l in func.inspect_asm(func.signatures[sig]).split('\n'):
        if keyword in l:
            count += 1
            print(l)
            if count >= limit:
                break
    if count == 0:
        print('No instructions found')

@jit(nopython=True)
def sqdiff(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        out[i] = (x[i] - y[i])**2
    return out

x32 = np.linspace(1, 2, 10000, dtype=np.float32)
y32 = np.linspace(2, 3, 10000, dtype=np.float32)
sqdiff(x32, y32)

x64 = x32.astype(np.float64)
y64 = y32.astype(np.float64)
sqdiff(x64, y64)

%timeit sqdiff(x32, y32)
%timeit sqdiff(x64, y64)

print('float32:')
find_instr(sqdiff, keyword='subp', sig=0)
print('---\nfloat64:')
find_instr(sqdiff, keyword='subp', sig=1)

## Other options
`@jit(nopython=True, fastmath=True)`
`find_instr(do_sum, keyword='mulp')`

In [ ]:
import sys
# sys.getsizeof(np.ones(3))
import scipy.sparse as sc
sc.csr_matrix(np.ones(3)).data.nbytes